In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
# we assume this is run on Colab
# you may need to mount your drive first
os.chdir("drive/My Drive/Colab Notebooks/2021")

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

def preprocess_images(images):
    return images.reshape(-1, 784).astype(np.float32) / 255

def preprocess_labels(labels):
    return labels.reshape(-1).astype(np.int32)

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)
train_labels = preprocess_labels(train_labels)
test_labels = preprocess_labels(test_labels)

train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(60000).batch(128).repeat()
#test_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(128)

In [ ]:
train_steps = 1000
lr = 0.1

W = tf.Variable(np.zeros([784, 10]).astype(np.float32))
b = tf.Variable(np.zeros(10, dtype=np.float32))

# first change: set up log dir and file writer(s)
from datetime import datetime
logdir = os.path.join("logs", "linear" + str(datetime.now()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))

In [ ]:
for step, (image_batch, label_batch) in enumerate(train_data):
    if step > train_steps:
        break

    with tf.GradientTape() as tape:
        logits = tf.matmul(image_batch, W) + b
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=label_batch))
        
    grads = tape.gradient(xent, [W, b])
    W.assign_sub(lr * grads[0])
    b.assign_sub(lr * grads[1])
    
    # change #2: log this stuff every time step (rather wasteful)
    with train_writer.as_default():
        tf.summary.scalar("loss", xent, step=step)
        tf.summary.histogram("logits", logits, step=step)
        tf.summary.histogram("weights", W, step=step)
    
    if not step % 100:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, label_batch), tf.float32))
        
        # change #3: log this only once every 100 steps
        with train_writer.as_default():
            tf.summary.scalar("accuracy", acc, step=step)
            tf.summary.image("input", tf.reshape(image_batch, [-1, 28, 28, 1]), step=step)
            
        test_preds = tf.argmax(tf.matmul(test_images, W) + b, axis=1, output_type=tf.int32)
        test_acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, test_labels), tf.float32))
        with test_writer.as_default():
            tf.summary.scalar("accuracy", test_acc, step=step)
        
        print("Training Loss: {} Accuracy: {}".format(xent, acc))
        print("Test accuracy: {}\n".format(test_acc))

In [ ]:
# then load/run tensorboard

%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs